In [9]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
# client.sample_mflix # 온점 표기법 -> 파일명 안에 온점이 있거나 숫자로 시작 -> 사용 불가
db = client["sample_mflix"] # 대괄호 표기법

users = db.users
comments = db.comments
movies = db.movies

print("ok", db.name, "collection :", [c for c in db.list_collection_names() if c in {"users", "comments", "movies"}])

ok sample_mflix collection : ['movies', 'comments', 'users']


In [14]:
loyal_users_pipeline = [
    {
        "$lookup": {
            "from": "comments",
            "localField": "email",
            "foreignField": "email",
            "as": "c"
        }
    },
    {
        "$addFields": {
            "commentsCount": {"$size": "$c"},
            "avgTextLen": {
                "$avg": {
                    "$map" : {
                        "input": "$c",
                        "as" : "x",
                        "in": {"$strLenCP": {"$ifNull": ["$$x.text", ""
                                                        ]}}
                    }
                }
            },
            "lastCommentDate": {"$max": "$c.date"}
        }
    },
    {"$match": {"commentsCount": {"$gte": 10}}},
    {"$sort": {"commentsCount": -1, "avgTextLen": -1, "lastCommentDate": -1}
]

loyal_users = list(users.aggregate(loyal_users_pipeline))
print(loyal_users[:2])
# NoSQL & SQL -> 현업 실무 개발자들은 JOIN -> 서로 다른 데이터 테이블 혹은 컬렉션을 하나로 연결해서 가져와 처리하려면 해당 데이터의 양이 방대해짐.
# 방대해진 데이터를 처리하기 위한 하드웨어 무리 -> 기계 성능 저하

[{'_id': ObjectId('59b99db4cfa9a34dcd7885b6'), 'name': 'Ned Stark', 'email': 'sean_bean@gameofthron.es', 'password': '$2b$12$UREFwsRUoyF0CRqGNK0LzO0HM/jLhgUCNNIJ9RJAqMUQ74crlJ1Vu', 'c': [{'_id': ObjectId('5a9427648b0beebeb6957b83'), 'name': 'Ned Stark', 'email': 'sean_bean@gameofthron.es', 'movie_id': ObjectId('573a1391f29313caabcd8243'), 'text': 'Illo nostrum enim sequi doloremque dolore saepe beatae. Iusto alias odit quaerat id dolores. Dolore quaerat accusantium esse voluptatibus. Aspernatur fuga exercitationem explicabo.', 'date': datetime.datetime(2000, 1, 21, 3, 17, 4)}, {'_id': ObjectId('5a9427648b0beebeb6957d08'), 'name': 'Ned Stark', 'email': 'sean_bean@gameofthron.es', 'movie_id': ObjectId('573a1391f29313caabcd96b9'), 'text': 'Architecto eos eum iste facilis. Sunt aperiam fugit nihil quas.', 'date': datetime.datetime(2005, 9, 24, 16, 22, 38)}, {'_id': ObjectId('5a9427648b0beebeb6957dec'), 'name': 'Ned Stark', 'email': 'sean_bean@gameofthron.es', 'movie_id': ObjectId('573a1392

In [ ]:
movies_report_Pipeline = [
    {"$facet": {
        "latest5": [
            {"$sort": {"released": -1}},
            {"$project": {"_id": "$title", "year": 1}},
            {"$limit": 5}
        ],
        "highRatedCount": [
            {"$match": {"imdb.rating": {"$gte": 8}}},
            {"$count": "HighRatedCount"}
        ],
        "genresTop10": [
            {"$unwind": "$genres"},
            {"$group": {"_id": "$genres", "movieCount": {"$sum": 1}}},
            {"$sort": {"movieCount": -1}},
            {"$limit": 10}
        ],
        "yearlyAvgRecent10": [
            {"$group": {"_id": "$year", "avgRating": {"$avg": "$imdb.rating"}}},
            {"$sort": {"_id": -1}},
            {"$limit": 10}
        ]
    }}
]

movies.aggregate(movies_report_Pipeline)